# Ames Housing Dataset Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Input

`AmesHousing.tsv` is from [here](https://jse.amstat.org/v19n3/decock/AmesHousing.txt).

Only replace empty cells with `NaN`.

In [ ]:
df_all_og = pd.read_csv('AmesHousing.tsv', sep='\t', na_values=[''], keep_default_na=False)

In [ ]:
df_all_og

## Dropping Columns

In [ ]:
df_all_og.drop(columns=['Order', 'PID', 'MS SubClass'], inplace=True)

## Renaming Columns

In [ ]:
columns = [
    "Zoning",
    "Lot Frontage",
    "Lot Area",
    "Street",
    "Alley",
    "Lot Shape",
    "Land Contour",
    "Utilities",
    "Lot Configuration",
    "Land Slope",
    "Neighborhood",
    "Condition 1",
    "Condition 2",
    "Building Type",
    "House Style",
    "Overall Quality",
    "Overall Condition",
    "Year Built",
    "Year Remodel",
    "Roof Style",
    "Roof Material",
    "Exterior 1st",
    "Exterior 2nd",
    "Masonry Veneer",
    "Masonry Veneer Area",
    "Exterior Quality",
    "Exterior Condition",
    "Foundation",
    "Basement Height",
    "Basement Condition",
    "Basement Exposure",
    "Basement Finish Rating 1",
    "Basement Finish Area 1",
    "Basement Finish Rating 2",
    "Basement Finish Area 2",
    "Basement Unfinished Area",
    "Total Basement Area",
    "Heating",
    "Heating Quality",
    "Central Air",
    "Electrical",
    "1st Floor Area",
    "2nd Floor Area",
    "Low Quality Finished Area",
    "Above Ground Living Area",
    "Basement Full Baths",
    "Basement Half Baths",
    "Full Baths",
    "Half Baths",
    "Bedrooms Above Ground",
    "Kitchens Above Ground",
    "Kitchen Quality",
    "Rooms Above Ground",
    "Functionality",
    "Fireplaces",
    "Fireplace Quality",
    "Garage Type",
    "Garage Year",
    "Garage Finish",
    "Garage Cars",
    "Garage Area",
    "Garage Quality",
    "Garage Condition",
    "Paved Driveway",
    "Wood Deck Area",
    "Open Porch Area",
    "Enclosed Porch Area",
    "3 Season Porch Area",
    "Screen Porch Area",
    "Pool Area",
    "Pool Quality",
    "Fence",
    "Misc Feature",
    "Misc Value",
    "Month Sold",
    "Year Sold",
    "Sale Type",
    "Sale Condition",
    "Sale Price"
]

Check that the new column names align with the old column names

In [ ]:
list(zip(df_all_og.columns, columns))

In [ ]:
df_all_og.columns = columns

In [ ]:
df_all_og

## Corrections

In [ ]:
df_all_og['Exterior 2nd'].unique()

Fix the impossible value for garage year.

In [ ]:
df_all_og['Garage Year'].describe()

In [ ]:
df_all_og.loc[df_all_og['Garage Year'] == 2207][['Garage Year', 'Year Built', 'Year Remodel']]

In [ ]:
df_all_og.loc[df_all_og['Garage Year'] == 2207, 'Garage Year'] = 2007

In [ ]:
df_all_og['Garage Year'].max()

Remove space after WD in the Sale Type feature.

In [ ]:
df_all_og['Sale Type'].unique()

In [ ]:
df_all_og.replace({ 'Sale Type': { 'WD ': 'WD' }}, inplace=True)

In [ ]:
df_all_og['Sale Type'].unique()

## Missing Values

In [ ]:
def get_na_counts(df):
    return sorted([(k, v) for k, v in dict(df.isna().sum()).items() if v > 0], key=lambda x: -x[1])

get_na_counts(df_all_og)

### Lot Frontage

Replace missing values with the median.

In [ ]:
df_all_og['Lot Frontage'].median()

In [ ]:
df_all_og['Lot Frontage'].fillna(df_all_og['Lot Frontage'].median(), inplace=True)

### Garage Year

If there is no garage year (like when there is no garage), then set it to the year the house was built.

In [ ]:
garage_is_na = df_all_og['Garage Year'].isna()
df_all_og.loc[garage_is_na, 'Garage Year'] = df_all_og[garage_is_na]['Year Built']

### Masonry Veneer and Masonry Veneer Area

Where there's no masonry veneer, set the area to 0 and the type to none.

In [ ]:
df_all_og['Masonry Veneer Area'].mode()

In [ ]:
df_all_og['Masonry Veneer Area'].fillna(0, inplace=True)
df_all_og['Masonry Veneer'].fillna('None', inplace=True)

### Other

Remove the remaining 9 instances that have missing values.

In [ ]:
get_na_counts(df_all_og)

In [ ]:
df_all_og[df_all_og.isna().sum(axis=1) > 0].shape

In [ ]:
df_all_og.dropna(inplace=True)

In [ ]:
df_all_og.shape

## One-hot Encoding

In [ ]:
one_hot_columns = [
    'Zoning',
    'Alley',
    'Land Contour',
    'Utilities',
    'Lot Configuration',
    'Neighborhood',
    'Condition 1',
    'Condition 2',
    'Building Type',
    'House Style',
    'Roof Style',
    'Roof Material',
    'Exterior 1st',
    'Exterior 2nd',
    'Masonry Veneer',
    'Foundation',
    'Basement Finish Rating 1',
    'Basement Finish Rating 2',
    'Heating',
    'Electrical',
    'Garage Type',
    'Sale Type',
    'Sale Condition',
    'Fence',
    'Misc Feature'
]

In [ ]:
df_all_one_hot = pd.get_dummies(df_all_og, columns=one_hot_columns, dtype=np.uint8)

In [ ]:
df_all_one_hot

## Ordinal encoding

In [ ]:
replace = {
    'Street': {
        'Grvl': 0,
        'Pave': 1
    },
    'Central Air': {
        'N': 0,
        'Y': 1
    },
    'Lot Shape': {
        'Reg': 0,
        'IR1': 1,
        'IR2': 2,
        'IR3': 3
    },
    'Land Slope': {
        'Gtl': 0,
        'Mod': 1,
        'Sev': 2
    },
    'Basement Height': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4,
        'NA': 5
    },
    'Basement Condition': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4,
        'NA': 5
    },
    'Basement Exposure': {
        'Gd': 0,
        'Av': 1,
        'Mn': 2,
        'No': 3,
        'NA': 4
    },
    'Exterior Quality': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4
    },
    'Exterior Condition': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4
    },
    'Heating Quality': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4
    },
    'Kitchen Quality': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4
    },
    'Functionality': {
        'Typ': 0,
        'Min1': 1,
        'Min2': 2,
        'Mod': 3,
        'Maj1': 4,
        'Maj2': 5,
        'Sev': 6,
        'Sal': 7
    },
    'Fireplace Quality': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4,
        'NA': 5
    },
    'Garage Finish': {
        'Fin': 0,
        'RFn': 1,
        'Unf': 2,
        'NA': 3
    },
    'Garage Quality': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4,
        'NA': 5
    },
    'Garage Condition': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'Po': 4,
        'NA': 5
    },
    'Paved Driveway': {
        'Y': 0,
        'P': 1,
        'N': 2
    },
    'Pool Quality': {
        'Ex': 0,
        'Gd': 1,
        'TA': 2,
        'Fa': 3,
        'NA': 4
    }
}

In [ ]:
df_all = df_all_one_hot.replace(to_replace=replace)

In [ ]:
df_all

## Output

In [ ]:
df_all

In [ ]:
df_train, df_test = train_test_split(df_all, test_size=0.25, random_state=0)

In [ ]:
df_train

In [ ]:
df_test.shape

In [ ]:
df_train.to_csv('AmesHousingPreprocessedTrain.csv', index=False)

In [ ]:
df_test.to_csv('AmesHousingPreprocessedTest.csv', index=False)